In [1]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from scipy import stats
from scipy import optimize
import joblib
import matplotlib.pyplot as plt

In [2]:
#Market and option parameters as in section 4.2 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
s_0 = 100
r = 0.01
corr= 0.3
tau = 1/52
T = 1/3
K = 100

mu = np.empty(20)
sigma = np.empty(20)
for i in range(0,10):
    mu[i] = mu[i+10] = (3+(i+1)/2)/100
    sigma[i] = sigma[i+10] = (15+(i+1))/100

cov_mat = np.empty((20,20))
for i in range(0,20):
    for j in range(0,20):
        if i != j:
            cov_mat[i,j] = corr
        else:
            cov_mat[i,j] = 1

C = np.linalg.cholesky(cov_mat)

#Confidence levels and parameters for GlueVaR
alpha_Glue = 0.95
beta_Glue = 0.995
omega_Glue = np.array([1/3,1/3])

In [3]:
M_MC = 10000000

In [4]:
#Function for calculating simulated values of S_tau and simulated payoffs P_T from simulations of multivariate standard normal random variables
def data_gen(M,N,Z,V):
    #correlating the independent components
    Z = np.transpose(np.matmul(C,np.transpose(Z)))
    V = np.transpose(np.matmul(C,np.transpose(V)))
    
    #simulate S_tau under P
    S_tau_pre = np.empty((M, 20))
    for j in range(0,20):
        S_tau_pre[:,j] = s_0 * np.exp( (mu[j]-0.5*sigma[j]**2)*tau + np.sqrt(tau)*sigma[j]*Z[:,j] )
    S_tau = np.tile(S_tau_pre, (N,1))

    #simulate S_T under Q
    S_T = np.empty((N*M,20))
    for j in range(0,20):
        S_T[:,j] = S_tau[:,j] * np.exp( (r-0.5*sigma[j]**2)*(T-tau) + np.sqrt(T-tau)*sigma[j]*V[:,j] )

    #compute discounted option payoffs
    P_T_pre =np.empty((len(S_T), 20))
    for j in range(0,10):
        P_T_pre[:,j] = np.exp(-r*(T-tau)) * np.maximum(S_T[:,j]-K,0)
    for j in range(10,20):
        P_T_pre[:,j] = np.exp(-r*(T-tau)) * np.maximum(K-S_T[:,j],0)
    P_T = np.sum(P_T_pre, axis=1)
    return S_tau,P_T

#the function DT(Z,\theta)
def data_trans_IS(Z,IS):
    res = np.empty((len(Z),20))
    for j in range(20):
        res[:,j] = Z[:,j]*np.sqrt(IS[20+j]) + IS[j]
    return res

#The density function corresponding to the physical probability measure P
def f(y):
    return stats.multivariate_normal.pdf(y, mean=np.full(20,0), cov=np.identity(20))

#The density function corresponding to the IS probability measure (note that x is interpreted as theta, needed for the least-squares solver to work properly)
def f_theta(y, x):
    return stats.multivariate_normal.pdf(y, mean=x[0:20], cov=np.diag(x[20:40]), allow_singular=True)

#Put and Call Black-Scholes Formulas
def d1(K, t, x, sigma, r):
    return (np.log(x/K)+(r+0.5*sigma**2)*t)/(sigma*np.sqrt(t))

def d2(K, t, x, sigma, r):
    return (np.log(x/K)+(r-0.5*sigma**2)*t)/(sigma*np.sqrt(t))

def put_true(K, t, x, sigma, r): 
    return K*np.exp(-r*t)*stats.norm.cdf(-d2(K,t,x,sigma,r)) - x*stats.norm.cdf(-d1(K,t,x,sigma,r))

def call_true(K, t, x, sigma, r):
    return x*stats.norm.cdf(d1(K,t,x,sigma,r)) - K*np.exp(-r*t)*stats.norm.cdf(d2(K,t,x,sigma,r))

#the true function l
def P_T_true(x):
    P_T = np.empty((len(x),20))
    for j in range(0,10):
        P_T[:,j] = call_true(K=100, t=(T-tau), x=x[:,j], sigma=sigma[j], r=r)
        P_T[:,j+10] = put_true(K=100, t=(T-tau), x=x[:,j+10], sigma=sigma[j+10], r=r)
    return np.sum(P_T, axis=1)

#This function describes the approximation of the expression inside the sum of m_2(theta)
def g_q_alpha_hat_reweighted(x,L,q_alpha_hat):
    return np.sqrt(f(y=L[:,0:20])/f_theta(y=L[:,0:20],x=x))*(L[:,-1]>q_alpha_hat)

#bounds for the IS density parameters (for the parameters corresponding to the mean no bounds are necessary, the standard deviation parameters however needs to be non-negative)
bnds_lower = np.empty((40))
bnds_upper = np.empty((40))
for j in range(20):
    bnds_lower[j] = -np.inf
    bnds_upper[j] = np.inf
    bnds_lower[20+j] = 0
    bnds_upper[20+j] = np.inf
    
bnds = (bnds_lower, bnds_upper)

#function for calculating GlueVaR
def GlueVaR(omega, L, alpha, beta):
    j_beta = int(len(L)*(1-beta))-1
    j_alpha = int(len(L)*(1-alpha))-1

    ES_beta = 1/(1-beta) * np.sum(L[0:j_beta-1])/len(L) + ( 1 - (j_beta-1)/((1-beta)*len(L)) )*L[j_beta]
    ES_alpha = 1/(1-alpha) * np.sum(L[0:j_alpha-1])/len(L) + ( 1 - (j_alpha-1)/((1-alpha)*len(L)) )*L[j_alpha]
    VaR_alpha = L[j_alpha]

    return omega[0]*ES_beta + omega[1]*ES_alpha + (1-omega[0]-omega[1])*VaR_alpha

In [8]:
#Generating realisations of standard normal random variables
Z = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_MC)
V = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_MC)

#Calculate the risk factor S_tau and the corresponding simulated payoffs P_T
S_tau, P_T = data_gen(M=M_MC, N=1, Z=Z, V=V)

#Calculate realisations of L_hat from the training data set using the trained neural network
L_hat = np.column_stack((Z, P_T_true(S_tau)))
L_hat_sort = L_hat[L_hat[:,-1].argsort()[::-1]]

#calculating GlueVaR estimator
GlueVaR_hat = GlueVaR(omega=omega_Glue, L=L_hat_sort[:,-1], alpha=alpha_Glue, beta=beta_Glue)

#reference value
print(GlueVaR_hat)

107.80585242434293
